In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
train['Date'] = pd.to_datetime(train['Date'], format='mixed')

train.info()

In [ ]:
unique_train_locations = train.groupby(['LAT', 'LON'])['GT_NO2'].mean().reset_index()
nans = train.loc[train['GT_NO2'].isna() == True, ['LAT', 'LON']].value_counts().to_frame().reset_index()
unique_train_locations = pd.merge(unique_train_locations, nans, on=['LAT', 'LON'], how='outer')
unique_train_locations.rename({'count': 'Nans'}, axis=1, inplace=True)
unique_train_locations['Nans'] = unique_train_locations['Nans'].apply(lambda x: round(x / 10.96, 2))

unique_train_locations

In [ ]:
ok_nans = unique_train_locations.loc[
    (unique_train_locations['Nans'] <= 5.0), 
    ['LAT', 'LON']
]

train = train.merge(ok_nans, on=['LAT', 'LON'], how='inner')

train.info()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf


groups = train.groupby(['LAT', 'LON'])

for name, group in groups:
    group.set_index('Date', inplace=True)
    plot_acf(x=group['GT_NO2'], lags=7, title=f'{name}', missing='drop')
    plt.show()